In [3]:
import pandas as pd
import numpy as np

from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate
from sklearn.metrics import f1_score, accuracy_score


Using TensorFlow backend.


In [4]:
df_train = pd.read_csv("C:/03 Master/FS22/MLHC/Project/archive/mitbih_train.csv", header=None)
df_train = df_train.sample(frac=1)
df_test = pd.read_csv("C:/03 Master/FS22/MLHC/Project/archive/mitbih_test.csv", header=None)

Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]


In [7]:

def get_model():
    nclass = 5
    inp = Input(shape=(187, 1))
    img_1 = Convolution1D(32, kernel_size=5, activation=activations.relu, padding="same")(inp)
    img_1 = Convolution1D(32, kernel_size=5, activation=activations.relu, padding="same")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(64, kernel_size=3, activation=activations.relu, padding="same")(img_1)
    img_1 = Convolution1D(64, kernel_size=3, activation=activations.relu, padding="same")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(64, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(64, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = GlobalMaxPool1D()(img_1)
    img_1 = Dropout(rate=0.2)(img_1)

    dense_1 = Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = Dense(nclass, activation=activations.softmax, name="dense_3_mitbih")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])
    model.summary()
    return model

In [6]:
model = get_model()
file_path = "baseline_cnn_mitbih.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 187, 1)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 187, 32)           192       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 187, 32)           5152      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 93, 32)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 93, 32)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 93, 64)            6208      
____________________

In [ ]:
model.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model.load_weights(file_path)

pred_test = model.predict(X_test)
pred_test = np.argmax(pred_test, axis=-1)

Train on 78798 samples, validate on 8756 samples
Epoch 1/1000
 - 109s - loss: 0.3251 - acc: 0.9045 - val_loss: 0.1821 - val_acc: 0.9464

Epoch 00001: val_acc improved from -inf to 0.94644, saving model to baseline_cnn_mitbih.h5
Epoch 2/1000
 - 106s - loss: 0.1835 - acc: 0.9482 - val_loss: 0.1356 - val_acc: 0.9653

Epoch 00002: val_acc improved from 0.94644 to 0.96528, saving model to baseline_cnn_mitbih.h5
Epoch 3/1000
 - 108s - loss: 0.1543 - acc: 0.9574 - val_loss: 0.1200 - val_acc: 0.9684

Epoch 00003: val_acc improved from 0.96528 to 0.96836, saving model to baseline_cnn_mitbih.h5
Epoch 4/1000
 - 108s - loss: 0.1374 - acc: 0.9621 - val_loss: 0.1157 - val_acc: 0.9680

Epoch 00004: val_acc did not improve from 0.96836
Epoch 5/1000
 - 114s - loss: 0.1270 - acc: 0.9645 - val_loss: 0.1082 - val_acc: 0.9696

Epoch 00005: val_acc improved from 0.96836 to 0.96962, saving model to baseline_cnn_mitbih.h5
Epoch 6/1000
 - 109s - loss: 0.1166 - acc: 0.9682 - val_loss: 0.1031 - val_acc: 0.9710



In [ ]:
f1 = f1_score(Y_test, pred_test, average="macro")

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)